In [1]:
import Model
import tensorflow as tf
import os
import re
import hashlib
import pandas as pd
import numpy as np
from extractMFCC import computeFeatures, computeFeatures1
from addNoise import addNoise
print(tf.__version__)

2.0.0-beta0


In [2]:
testWAVs = pd.read_csv('Dataset/testing_list.txt', sep=" ", header=None)[0].tolist()
valWAVs  = pd.read_csv('Dataset/validation_list.txt', sep=" ", header=None)[0].tolist()

for idx, item in enumerate(testWAVs):
    testWAVs[idx] = "Dataset/" + testWAVs[idx]

for idx, item in enumerate(valWAVs):
    valWAVs[idx] = "Dataset/" + valWAVs[idx]

In [3]:
DictCategs = {'nine' : 1, 'yes' : 2, 
                         'no' : 3, 'up' : 4, 'down' : 5, 'left' : 6, 'right' : 7, 'on' : 8, 'off' : 9, 'stop' : 10, 'go' : 11,
                         'zero' : 12, 'one' : 13, 'two' : 14, 'three' : 15, 'four' : 16, 'five' : 17, 'six' : 18, 
                         'seven' : 19,  'eight' : 20, 'backward':0, 'bed':0, 'bird':0, 'cat':0, 'dog':0,
                         'follow':0, 'forward':0, 'happy':0, 'house':0, 'learn':0, 'marvin':0, 'sheila':0, 'tree':0,
                         'visual':0, 'wow':0}
nCategs = 21

In [4]:
allWAVs  = []
for root, dirs, files in os.walk('Dataset/'):
    if root != "Dataset/_background_noise_":
        allWAVs += [root+'/'+ f for f in files if f.endswith('.wav')]
trainWAVs = list( set(allWAVs)-set(valWAVs)-set(testWAVs) )

In [5]:
print(len(trainWAVs))
print(len(valWAVs))
print(len(testWAVs))

84843
9981
11005


In [6]:
def _getFileCategory(file, catDict):
    """
    Receives a file with name Dataset/<cat>/<filename> and returns an integer that is catDict[cat]
    """
    categ = os.path.basename(os.path.dirname(file))
    return catDict.get(categ,0)

In [7]:
#get categories
testWAVlabels = [_getFileCategory(f, DictCategs) for f in testWAVs]
valWAVlabels = [_getFileCategory(f, DictCategs) for f in valWAVs]
trainWAVlabels = [_getFileCategory(f, DictCategs) for f in trainWAVs]

In [8]:
print(len(trainWAVlabels))
print(len(valWAVlabels))
print(len(testWAVlabels))

84843
9981
11005


In [ ]:
train = np.zeros((len(trainWAVlabels), 99, 39))
for i in range(len(trainWAVs)):
    #print(i)
    train[i, :, :] = computeFeatures(trainWAVs[i])
#prova = computeFeatures(trainWAVs[0])
#print(prova.shape)

In [9]:
#np.save('train.npy', train)
train = np.load('train.npy')

In [10]:
print(train.shape)

model = Model.AttentionModel(21, 99, 39, use_GRU = True)
model.summary()

(84843, 99, 39)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 39, 99, 1)]  0                                            
__________________________________________________________________________________________________
permute (Permute)               (None, 99, 39, 1)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 99, 39, 10)   60          permute[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 99, 39, 10)   40          conv2d[0][0]                     
______________________________________________________________________________

In [13]:
train = train.reshape(train.shape[0], 99, 39, 1)

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

results = model.fit(x = train, y = trainWAVlabels, batch_size = 32, epochs = 1, use_multiprocessing = True, workers = 4)

ValueError: Error when checking input: expected input_1 to have shape (39, 99, 1) but got array with shape (99, 39, 1)